In [1]:
import json

import pandas as pd

from ut.gsdf import gc, set_ss_with_dataframe

file_path = "events_log_MZc25181b2b14dde4fa5af27b462fc421e.jsonl"
with open(file_path) as f:
  lines = f.readlines()
  jsons = [json.loads(line) for line in lines]
  df = pd.json_normalize(jsons)

twilio_df = df[df["meta.from"] == "twilio"]
aoai_df = df[df["meta.from"] == "aoai"]

grouped = twilio_df.groupby(twilio_df["event"])
event_dfs = {f"twilio.{event}": group for event, group in grouped}

grouped = aoai_df.groupby(aoai_df["type"])
event_dfs.update({f"aoai.{event}": group for event, group in grouped})

# drop empty columns generated by json_normalize
event_dfs = {name: df.dropna(axis=1, how="all") for name, df in event_dfs.items()}

ss = gc.open("realtime log")
set_ss_with_dataframe(ss, event_dfs, clear=True)

 94%|█████████▍| 17/18 [02:01<00:07,  7.21s/it]/Users/soukun/botchan-ai-devkit/realtime_audio/ut/gsdf.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna("")
100%|██████████| 18/18 [02:08<00:00,  7.16s/it]
